In [1]:
import tensorflow as tf
import numpy as np
import math
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Remove previous Tensors and Operations
tf.reset_default_graph()

In [3]:
mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# Parameters
learning_rate = 0.001
n_epochs = 100
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1
save_file = './train_model.ckpt'

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

In [5]:
n_hidden_layer = 256 # layer number of features

In [6]:
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
# tf Graph input
features = tf.placeholder("float", [None, 28, 28, 1])
labels = tf.placeholder("float", [None, n_classes])

# None - Placeholder for Batch Size
# 28x28 - Image size
# 1 - Channel Size (RGB has 3 channels, Grayscale has 1)

features_flat = tf.reshape(features, [-1, n_input])
# reshapes the 28px by 28px matrices in x into row vectors of 784px.

In [9]:
# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(features_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

In [10]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [11]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
# Initializing the variables
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})
        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
    
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')
    
    

# The MNIST library in TensorFlow provides the ability to receive the dataset in batches. 
# Calling the mnist.train.next_batch() function returns a subset of the training data.

# batch_features.shape = (128, 28, 28, 1)
# batch_labels.shape = (128, 10)

Epoch 0   - Validation Accuracy: 0.3424000144004822
Epoch 10  - Validation Accuracy: 0.7851999998092651
Epoch 20  - Validation Accuracy: 0.8253999948501587
Epoch 30  - Validation Accuracy: 0.8456000089645386
Epoch 40  - Validation Accuracy: 0.8582000136375427
Epoch 50  - Validation Accuracy: 0.8672000169754028
Epoch 60  - Validation Accuracy: 0.8718000054359436
Epoch 70  - Validation Accuracy: 0.8769999742507935
Epoch 80  - Validation Accuracy: 0.8812000155448914
Epoch 90  - Validation Accuracy: 0.885200023651123
Trained Model Saved.


In [13]:
# Load a Trained Model
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})

print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.8867999911308289
